In [27]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import pickle
import time
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load my own functions
#import dnnregressor_train_eval_keras as dnnk
#import make_data_wfpt as mdw
from kde_training_utilities import kde_load_data
import ddm_data_simulation as ddm_sim
import boundary_functions as bf

ModuleNotFoundError: No module named 'kde_training_utilities'

In [2]:
import yaml

In [21]:
with open('/media/data_cifs/afengler/git_repos/nn_likelihoods/kde_mle_parallel.yaml', 'r') as stream:
    data_loaded = yaml.unsafe_load(stream)

#yaml.safe_load('/media/data_cifs/afengler/tmp/my_yaml_test.yaml')

In [23]:
data_loaded['param_bounds']

[(-1, 1), (0.5, 2), (0.3, 0.7)]

In [29]:
import os
os.getcwd()

'/media/data_cifs/afengler/git_repos/nn_likelihoods/tmp'

In [2]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5069085281283002902
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17258515983273140430
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12048773940
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12938403459843521874
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:03:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7595569028926069205
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
# Load Model
model_path = '/media/data_cifs/afengler/data/kde/linear_collapse/keras_models/dnnregressor_ddm_linear_collapse_06_22_19_23_27_28/model_0' 
ckpt_path = '/media/data_cifs/afengler/data/kde/linear_collapse/keras_models/dnnregressor_ddm_linear_collapse_06_22_19_23_27_28/ckpt_0_130'

model = keras.models.load_model(model_path)
model.load_weights(ckpt_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
model.predict()

In [4]:
# Initializations -----
n_runs = 100
n_samples = 2500
feature_file_path = '/media/data_cifs/afengler/data/kde/linear_collapse/train_test_data/test_features.pickle'
mle_out_path = '/media/data_cifs/afengler/data/kde/linear_collapse/mle_runs'

# NOTE PARAMETERS: WEIBULL: [v, a, w, node, shape, scale]
param_bounds = [(-1, 1), (0.3, 2), (0.3, 0.7), (0.01, 0.01), (0, np.pi / 2.2)]


my_optim_columns = ['v_sim', 'a_sim', 'w_sim', 'node_sim', 'theta_sim',
                    'v_mle', 'a_mle', 'w_mle', 'node_mle', 'theta_mle', 'n_samples']

# Get parameter names in correct ordering:
dat = pickle.load(open(feature_file_path, 
                       'rb'))

parameter_names = list(dat.keys())[:-2] # :-1 to get rid of 'rt' and 'choice' here

# Make columns for optimizer result table
p_sim = []
p_mle = []

for parameter_name in parameter_names:
    p_sim.append(parameter_name + '_sim')
    p_mle.append(parameter_name + '_mle')
    
my_optim_columns = p_sim + p_mle + ['n_samples']

# Initialize the data frame in which to store optimizer results
optim_results = pd.DataFrame(np.zeros((n_runs, len(my_optim_columns))), columns = my_optim_columns)
optim_results.iloc[:, 2 * len(parameter_names)] = n_samples

# define boundary
boundary = bf.linear_collapse
boundary_multiplicative = False

# Define the likelihood function
def log_p(params = [0, 1, 0.9], model = [], data = [], parameter_names = []):
    # Make feature array
    feature_array = np.zeros((data[0].shape[0], len(parameter_names) + 2))
    
    # Store parameters
    cnt = 0
    for i in range(0, len(parameter_names), 1):
        feature_array[:, i] = params[i]
        cnt += 1
    
    # Store rts and choices
    feature_array[:, cnt] = data[0].ravel() # rts
    feature_array[:, cnt + 1] = data[1].ravel() # choices
    
    # Get model predictions
    prediction = model.predict(feature_array)
    
    # Some post-processing of predictions
    prediction[prediction < 1e-29] = 1e-29
    
    return(- np.sum(np.log(prediction)))  

def make_params(param_bounds = []):
    params = np.zeros(len(param_bounds))
    
    for i in range(len(params)):
        params[i] = np.random.uniform(low = param_bounds[i][0], high = param_bounds[i][1])
        
    return params
# ---------------------

In [5]:
# Main loop ----------- TD: Parallelize
for i in range(0, n_runs, 1): 
    
    # Get start time
    start_time = time.time()
    
#     # Sample parameters
#     v_sim = np.random.uniform(high = v_range[1], low = v_range[0])
#     a_sim = np.random.uniform(high = a_range[1], low = a_range[0])
#     w_sim = np.random.uniform(high = w_range[1], low = w_range[0])

#     #c1_sim = np.random.uniform(high = c1_range[1], low = c1_range[0])
#     #c2_sim = np.random.uniform(high = c2_range[1], low = c2_range[0])
#     node_sim = np.random.uniform(high = node_range[1], low = node_range[0])
#     shape_sim = np.random.uniform(high = shape_range[1], low = shape_range[0])
#     scale_sim = np.random.uniform(high = scale_range[1], low = scale_range[0])

    tmp_params = make_params(param_bounds = param_bounds)
    
    # Store in output file
    optim_results.iloc[i, :len(parameter_names)] = tmp_params
    
    # Print some info on run
    print('Parameters for run ' + str(i) + ': ')
    print(tmp_params)
    
    # Define boundary params
    boundary_params = {'node': tmp_params[3],
                       'theta': tmp_params[4]}
    
    # Run model simulations
    ddm_dat_tmp = ddm_sim.ddm_flexbound_simulate(v = tmp_params[0],
                                                 a = tmp_params[1],
                                                 w = tmp_params[2],
                                                 s = 1,
                                                 delta_t = 0.001,
                                                 max_t = 20,
                                                 n_samples = n_samples,
                                                 boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                                 boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                                 boundary_params = boundary_params)
        
    # Print some info on run
    print('Mean rt for current run: ')
    print(np.mean(ddm_dat_tmp[0]))
    
    # Run optimizer
    out = differential_evolution(log_p, 
                                 bounds = param_bounds, 
                                 args = (model, ddm_dat_tmp, parameter_names), 
                                 popsize = 30,
                                 disp = True)
    
    # Print some info
    print('Solution vector of current run: ')
    print(out.x)
    
    print('The run took: ')
    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
    # Store result in output file
    optim_results.iloc[i, len(parameter_names):(2*len(parameter_names))] = out.x
# -----------------------

# Save optimization results to file
optim_results.to_csv(mle_out_path + '/mle_results_1.csv')

Parameters for run 0: 
[0.07249486 0.30623271 0.66737066 0.01       0.33118348]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.08035040000000006
differential_evolution step 1: f(x)= -1961.56
differential_evolution step 2: f(x)= -1964.23
differential_evolution step 3: f(x)= -2167.71
differential_evolution step 4: f(x)= -2167.71
differential_evolution step 5: f(x)= -2324.47
differential_evolution step 6: f(x)= -2324.47
differential_evolution step 7: f(x)= -2324.47
differential_evolution step 8: f(x)= -2324.47
differential_evolution step 9: f(x)= -2343.2
differential_evolution step 10: f(x)= -2343.2
differential_evolution step 11: f(x)= -2343.2
differential_evolution step 12: f(x)= -2355.01
differential_evolution step 13: f(x)= -2355.01
differential_evolution step 14: f(x)= -2359.62
differential_evolution step 15: f(x)= -2359.62
differential_evolution step 16: f(x)= -2359.62
differential_evolution step 17: f(x)= -2359.62
differential_ev

/home/afengler/miniconda3/envs/pytorch/lib/python3.7/site-packages/scipy/optimize/_differentialevolution.py:901: RuntimeWarning: invalid value encountered in true_divide
  return (parameters - self.__scale_arg1) / self.__scale_arg2 + 0.5


Solution vector of current run: 
[-0.00562712  1.95970631  0.47784762  0.01        0.96885033]
The run took: 
00:04:42
Parameters for run 3: 
[0.94530018 0.30792041 0.4324437  0.01       0.96150267]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.06418000000000004
differential_evolution step 1: f(x)= -2964.13
differential_evolution step 2: f(x)= -3099.12
differential_evolution step 3: f(x)= -3099.12
differential_evolution step 4: f(x)= -3099.12
differential_evolution step 5: f(x)= -3189.06
differential_evolution step 6: f(x)= -3267.82
differential_evolution step 7: f(x)= -3267.82
differential_evolution step 8: f(x)= -3267.82
differential_evolution step 9: f(x)= -3267.82
differential_evolution step 10: f(x)= -3275.24
differential_evolution step 11: f(x)= -3288.89
differential_evolution step 12: f(x)= -3288.89
differential_evolution step 13: f(x)= -3288.89
differential_evolution step 14: f(x)= -3311.31
differential_evolution step 15: f(

differential_evolution step 1: f(x)= -764.272
differential_evolution step 2: f(x)= -940.315
differential_evolution step 3: f(x)= -943.492
differential_evolution step 4: f(x)= -943.492
differential_evolution step 5: f(x)= -1012
differential_evolution step 6: f(x)= -1012
differential_evolution step 7: f(x)= -1025.97
differential_evolution step 8: f(x)= -1025.97
differential_evolution step 9: f(x)= -1025.97
differential_evolution step 10: f(x)= -1025.97
differential_evolution step 11: f(x)= -1030.93
differential_evolution step 12: f(x)= -1030.93
differential_evolution step 13: f(x)= -1030.93
differential_evolution step 14: f(x)= -1035.18
differential_evolution step 15: f(x)= -1035.47
differential_evolution step 16: f(x)= -1035.47
differential_evolution step 17: f(x)= -1039.27
differential_evolution step 18: f(x)= -1039.91
differential_evolution step 19: f(x)= -1039.91
differential_evolution step 20: f(x)= -1039.91
differential_evolution step 21: f(x)= -1041.28
differential_evolution step 

differential_evolution step 7: f(x)= 3700.09
differential_evolution step 8: f(x)= 3680.81
differential_evolution step 9: f(x)= 3659.95
differential_evolution step 10: f(x)= 3659.95
differential_evolution step 11: f(x)= 3656.4
differential_evolution step 12: f(x)= 3656.4
differential_evolution step 13: f(x)= 3650.64
differential_evolution step 14: f(x)= 3645.6
differential_evolution step 15: f(x)= 3645.6
differential_evolution step 16: f(x)= 3645.6
differential_evolution step 17: f(x)= 3636.61
differential_evolution step 18: f(x)= 3636.61
differential_evolution step 19: f(x)= 3636.56
differential_evolution step 20: f(x)= 3633.71
differential_evolution step 21: f(x)= 3633.71
Solution vector of current run: 
[-0.9563686   1.13053111  0.64043638  0.01        0.45724156]
The run took: 
00:05:01
Parameters for run 16: 
[0.79091382 1.66410137 0.57639861 0.01       1.21442918]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.3718540000000003
d

...

differential_evolution step 14: f(x)= -758.669
differential_evolution step 15: f(x)= -758.669
differential_evolution step 16: f(x)= -758.669
differential_evolution step 17: f(x)= -758.669
differential_evolution step 18: f(x)= -758.669
differential_evolution step 19: f(x)= -758.669
differential_evolution step 20: f(x)= -758.669
differential_evolution step 21: f(x)= -758.669
differential_evolution step 22: f(x)= -759.422
differential_evolution step 23: f(x)= -759.422
differential_evolution step 24: f(x)= -759.422
differential_evolution step 25: f(x)= -759.655
differential_evolution step 26: f(x)= -760.094
differential_evolution step 27: f(x)= -760.094
differential_evolution step 28: f(x)= -760.094
differential_evolution step 29: f(x)= -760.094
Solution vector of current run: 
[-0.07041266  0.58237812  0.67963046  0.01        0.74086333]
The run took: 
00:05:58
Parameters for run 76: 
[0.87581338 1.38302293 0.30714852 0.01       0.13042371]
0  datapoints sampled
1000  datapoints sampled
2

differential_evolution step 14: f(x)= 2696.59
differential_evolution step 15: f(x)= 2696.59
differential_evolution step 16: f(x)= 2694.19
differential_evolution step 17: f(x)= 2694.19
differential_evolution step 18: f(x)= 2694.19
differential_evolution step 19: f(x)= 2694.19
differential_evolution step 20: f(x)= 2694.19
differential_evolution step 21: f(x)= 2693.17
differential_evolution step 22: f(x)= 2693.17
differential_evolution step 23: f(x)= 2692.69
differential_evolution step 24: f(x)= 2692.69
differential_evolution step 25: f(x)= 2692.69
differential_evolution step 26: f(x)= 2692.58
differential_evolution step 27: f(x)= 2692.17
differential_evolution step 28: f(x)= 2692.17
Solution vector of current run: 
[0.35964829 1.99709202 0.49555635 0.01       0.99651608]
The run took: 
00:06:13
Parameters for run 82: 
[0.96267449 1.04471247 0.58808257 0.01       1.13326736]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.243727600000000

differential_evolution step 6: f(x)= 4182.81
differential_evolution step 7: f(x)= 4182.81
differential_evolution step 8: f(x)= 4179.55
differential_evolution step 9: f(x)= 4155.68
differential_evolution step 10: f(x)= 4152.2
differential_evolution step 11: f(x)= 4152.2
differential_evolution step 12: f(x)= 4152.2
differential_evolution step 13: f(x)= 4149.21
differential_evolution step 14: f(x)= 4148.38
Solution vector of current run: 
[0.10580396 1.79256342 0.49011251 0.01       0.454429  ]
The run took: 
00:04:08
Parameters for run 88: 
[-0.98011173  1.47262814  0.64248624  0.01        0.25917173]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
1.1663467999999648
differential_evolution step 1: f(x)= 3327.91
differential_evolution step 2: f(x)= 3235.49
differential_evolution step 3: f(x)= 3099.7
differential_evolution step 4: f(x)= 3099.7
differential_evolution step 5: f(x)= 3074.13
differential_evolution step 6: f(x)= 3021.3
different

1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.516588399999998
differential_evolution step 1: f(x)= 1583.67
differential_evolution step 2: f(x)= 1573.41
differential_evolution step 3: f(x)= 1573.41
differential_evolution step 4: f(x)= 1573.41
differential_evolution step 5: f(x)= 1573.41
differential_evolution step 6: f(x)= 1573.41
differential_evolution step 7: f(x)= 1573.41
differential_evolution step 8: f(x)= 1573.41
differential_evolution step 9: f(x)= 1573.41
differential_evolution step 10: f(x)= 1573.41
differential_evolution step 11: f(x)= 1561.27
differential_evolution step 12: f(x)= 1556.21
differential_evolution step 13: f(x)= 1556.21
differential_evolution step 14: f(x)= 1556.21
differential_evolution step 15: f(x)= 1556.21
differential_evolution step 16: f(x)= 1556.21
differential_evolution step 17: f(x)= 1553.73
differential_evolution step 18: f(x)= 1553.73
differential_evolution step 19: f(x)= 1553.61
differential_evolution step 20: f(x)= 155

differential_evolution step 3: f(x)= -3061.11
differential_evolution step 4: f(x)= -3191.24
differential_evolution step 5: f(x)= -3210.99
differential_evolution step 6: f(x)= -3210.99
differential_evolution step 7: f(x)= -3210.99
differential_evolution step 8: f(x)= -3267.71
differential_evolution step 9: f(x)= -3267.71
differential_evolution step 10: f(x)= -3319.13
differential_evolution step 11: f(x)= -3319.13
differential_evolution step 12: f(x)= -3319.13
differential_evolution step 13: f(x)= -3319.13
differential_evolution step 14: f(x)= -3319.13
differential_evolution step 15: f(x)= -3319.13
differential_evolution step 16: f(x)= -3319.13
differential_evolution step 17: f(x)= -3319.13
differential_evolution step 18: f(x)= -3319.87
differential_evolution step 19: f(x)= -3331.84
differential_evolution step 20: f(x)= -3337.71
differential_evolution step 21: f(x)= -3337.71
differential_evolution step 22: f(x)= -3337.71
differential_evolution step 23: f(x)= -3337.71
differential_evoluti

In [ ]:
# Read in results
optim_results = pd.read_csv(os.getcwd() + '/experiments/ddm_flexbound_kde_mle_fix_v_0_c1_0_w_unbiased_arange_2_3/optim_results.csv')

In [ ]:
plt.scatter(optim_results['v_sim'], optim_results['v_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for v
reg = LinearRegression().fit(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))
reg.score(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))

In [ ]:
plt.scatter(optim_results['a_sim'], optim_results['a_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for a
reg = LinearRegression().fit(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))
reg.score(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))

In [ ]:
plt.scatter(optim_results['w_sim'], optim_results['w_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))
reg.score(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))

In [ ]:
plt.scatter(optim_results['c1_sim'], optim_results['c1_mle'])

In [ ]:
# Regression for c1
reg = LinearRegression().fit(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))
reg.score(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))

In [ ]:
plt.scatter(optim_results['c2_sim'], optim_results['c2_mle'], c = optim_results['a_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))
reg.score(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))